In this notebook we clean and produce several datasets from the paper [In Silico Prediction of Cytochrome P450-Drug Interaction: QSARs for CYP3A4 and CYP2C9](https://www.mdpi.com/1422-0067/17/6/914). The final datasets will have CanoncialSMILES and inchikeys from the molecules in the clean dataset.

In [1]:
import time
import requests
import numpy as np
import pandas as pd
from rdkit import Chem
from io import StringIO

In [2]:
data = pd.read_csv("../datasets/CYP/AID_1851_datatable_all.csv")

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (0,7,9,10,12,13,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,42,43,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,66,67,69,70,71,72,73,74,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,93,94,96,97,98,99,100,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,120,121,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def rename_cols(col_name, data):
    if col_name.startswith("Activity "):
        return data[col_name][1]
    else:
        return col_name

After loading the dataset, we remove some roes of the dataset that contain a descriptions of the columns and rename the activity and score columns to indicate to which isoform they correspond

In [4]:
new_cols = [rename_cols(col, data) for col in data.columns]
data.columns = new_cols
to_rm = data["PUBCHEM_RESULT_TAG"].str.startswith("RESULT")
to_rm[to_rm.isnull()] = False
data = data[~to_rm.astype(bool)]

In [5]:
data = data.drop(columns="PUBCHEM_RESULT_TAG")
data = data.reset_index(drop=True)

In [6]:
def filter_cols(col_name):
    return " Activity Outcome" in col_name or " Activity Score" in col_name

We also drop columns that we will not used to simplify the data frame

In [7]:
a = [col for col in data.columns if filter_cols(col)]
columns_to_keep = ["PUBCHEM_SID", "PUBCHEM_CID", "Inhibition Observed"]+a
columns_to_remove = set(data.columns)-set(columns_to_keep)
data = data.drop(columns=columns_to_remove)

In [8]:
data.columns

Index(['PUBCHEM_SID', 'PUBCHEM_CID', 'Inhibition Observed',
       'p450-cyp2c19 Activity Outcome', 'p450-cyp2c19 Activity Score',
       'p450-cyp2d6 Activity Outcome', 'p450-cyp2d6 Activity Score',
       'p450-cyp3a4 Activity Outcome', 'p450-cyp3a4 Activity Score',
       'p450-cyp1a2 Activity Outcome', 'p450-cyp1a2 Activity Score',
       'p450-cyp2c9 Activity Outcome', 'p450-cyp2c9 Activity Score'],
      dtype='object')

We set appropiate types for some of the columns, and drop rows with missing values

In [9]:
data["Inhibition Observed"] = data["Inhibition Observed"].astype(str)

In [10]:
data = data.dropna()
data["Inhibition Observed"] = data["Inhibition Observed"].str.upper()

In [11]:
data["PUBCHEM_SID"] = data["PUBCHEM_SID"].astype(int)
data["PUBCHEM_CID"] = data["PUBCHEM_CID"].astype(int)
data

,PUBCHEM_SID,PUBCHEM_CID,Inhibition Observed,p450-cyp2c19 Activity Outcome,p450-cyp2c19 Activity Score,p450-cyp2d6 Activity Outcome,p450-cyp2d6 Activity Score,p450-cyp3a4 Activity Outcome,p450-cyp3a4 Activity Score,p450-cyp1a2 Activity Outcome,p450-cyp1a2 Activity Score,p450-cyp2c9 Activity Outcome,p450-cyp2c9 Activity Score
0,842238,6602638,TRUE,Inactive,0,Inconclusive,20,Inactive,0,Inactive,0,Inconclusive,20
1,842250,644510,TRUE,Inconclusive,20,Inconclusive,21,Active,85,Active,42,Inconclusive,20
2,842319,1960010,TRUE,Inconclusive,20,Inactive,0,Inconclusive,22,Active,85,Active,87
3,842408,644675,TRUE,Active,90,Inactive,0,Active,41,Active,41,Inconclusive,20
4,842584,644851,TRUE,Active,41,Inconclusive,20,Inconclusive,20,Active,86,Active,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17138,26751437,16758815,TRUE,Inconclusive,10,Inactive,0,Inactive,0,Inactive,0,Inactive,0
17139,26751438,16758816,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0
17140,26751439,16758817,TRUE,Active,44,Inactive,0,Inactive,0,Inactive,0,Inconclusive,20
17141,26751440,16758818,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0


The next thing we need to do is to deal with compounds that are duplicated. What we will do is to check wether the duplicates contain consistent information, that is if they are classified as active or incative for the same isoforms. If they are not, we discard all of them, if they are we, arbitrarely, keep the first one, as all contain the same information

In [12]:
duplicated_values = data[data.duplicated(subset=["PUBCHEM_CID"], keep=False)]
duplicated_values

,PUBCHEM_SID,PUBCHEM_CID,Inhibition Observed,p450-cyp2c19 Activity Outcome,p450-cyp2c19 Activity Score,p450-cyp2d6 Activity Outcome,p450-cyp2d6 Activity Score,p450-cyp3a4 Activity Outcome,p450-cyp3a4 Activity Score,p450-cyp1a2 Activity Outcome,p450-cyp1a2 Activity Score,p450-cyp2c9 Activity Outcome,p450-cyp2c9 Activity Score
283,860967,5775,TRUE,Active,40,Active,65,Inactive,0,Inactive,0,Inactive,0
4913,4252508,307957,TRUE,Inactive,0,Inactive,0,Inconclusive,21,Inactive,0,Inactive,0
5078,4252678,88779,TRUE,Active,48,Active,45,Active,43,Active,91,Active,46
5106,4252707,3246080,TRUE,Inactive,0,Active,89,Inactive,0,Inactive,0,Inactive,0
5300,4252906,2915,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17120,26751418,16758799,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0
17121,26751419,16758800,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0
17122,26751420,16758799,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0
17123,26751421,16758799,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0


In [13]:
def is_consistent(duplicates):
    """
        Check if duplicates are consistent with respect to the activity measured
    """
    return len(duplicates["p450-cyp3a4 Activity Outcome"].unique()) == 1 and len(duplicates["p450-cyp2c9 Activity Outcome"].unique()) == 1

In [14]:
to_filter = []
for cid in duplicated_values["PUBCHEM_CID"].unique():
    repeated = duplicated_values[duplicated_values["PUBCHEM_CID"] == cid]
    if not is_consistent(repeated):
        to_filter.extend(repeated.index)
    else:
        # if they are consistent, just keep the first one, the rest will be the same
        to_filter.extend(repeated.index[1:])

In [15]:
data = data.drop(index=to_filter)

In [16]:
data

,PUBCHEM_SID,PUBCHEM_CID,Inhibition Observed,p450-cyp2c19 Activity Outcome,p450-cyp2c19 Activity Score,p450-cyp2d6 Activity Outcome,p450-cyp2d6 Activity Score,p450-cyp3a4 Activity Outcome,p450-cyp3a4 Activity Score,p450-cyp1a2 Activity Outcome,p450-cyp1a2 Activity Score,p450-cyp2c9 Activity Outcome,p450-cyp2c9 Activity Score
0,842238,6602638,TRUE,Inactive,0,Inconclusive,20,Inactive,0,Inactive,0,Inconclusive,20
1,842250,644510,TRUE,Inconclusive,20,Inconclusive,21,Active,85,Active,42,Inconclusive,20
2,842319,1960010,TRUE,Inconclusive,20,Inactive,0,Inconclusive,22,Active,85,Active,87
3,842408,644675,TRUE,Active,90,Inactive,0,Active,41,Active,41,Inconclusive,20
4,842584,644851,TRUE,Active,41,Inconclusive,20,Inconclusive,20,Active,86,Active,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17138,26751437,16758815,TRUE,Inconclusive,10,Inactive,0,Inactive,0,Inactive,0,Inactive,0
17139,26751438,16758816,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0
17140,26751439,16758817,TRUE,Active,44,Inactive,0,Inactive,0,Inactive,0,Inconclusive,20
17141,26751440,16758818,FALSE,Inactive,0,Inactive,0,Inactive,0,Inactive,0,Inactive,0


Once we have a clean data set, we download from Pubchem the Canonical smiles and InChI Key, that will allow us to construct the molecule to extract possible descriptors

In [17]:
def list_string(l):
    return ",".join(map(str, l))

def get_smiles(data_f, window_size=300, wait_time=0.5):
    new_df = None
    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/%s/property/CanonicalSMILES,InChI/CSV"
    ids = data_f["PUBCHEM_CID"]
    n = len(ids)
    for i in range(int(np.ceil(n/window_size))):
        if i % 10 == 0:
            print ("Round", i)
        r = requests.get(url % list_string(ids[i*window_size:(i+1)*window_size]))
        if new_df is None:
            new_df = pd.read_csv(StringIO(r.text))
        else:
            new_df = pd.concat([new_df, pd.read_csv(StringIO(r.text))], ignore_index=True)
        time.sleep(wait_time)
    return new_df
        

In [18]:
smiles = get_smiles(data)

Round 0
Round 10
Round 20
Round 30
Round 40
Round 50


We incorporate the molecular data into the data set

In [19]:
data = data.merge(smiles, how="inner", left_on="PUBCHEM_CID", right_on="CID")
data = data.drop(columns="CID")

We split the data set into three: one that contains the shared molecules between the 2c9 and 3a4 isoforms and two containing the molecules exclusive to each

In [20]:
mask_3a4 = data["p450-cyp3a4 Activity Outcome"] != "Inconclusive"
mask_2c9 = data["p450-cyp2c9 Activity Outcome"] != "Inconclusive"
xor_masks = mask_2c9 ^ mask_3a4
only_3a4 = mask_3a4 & (mask_3a4 ^ mask_2c9)
only_2c9 = mask_2c9 & (mask_3a4 ^ mask_2c9)

In [21]:
print("Molecules in the shared data set", sum(mask_3a4 & mask_2c9))
print("Molecules in the 3a4 only data set", sum(only_3a4))
print("Molecules in the 2c9 only data set", sum(only_2c9))

Molecules in the shared data set 9432
Molecules in the 3a4 only data set 2995
Molecules in the 2c9 only data set 2813


In [40]:
data_shared = data[mask_3a4 & mask_2c9]
data_shared = data_shared.reset_index(drop=True)
data_only_3a4 = data[only_3a4]
data_only_3a4 = data_only_3a4.reset_index(drop=True)
data_only_2c9 = data[only_2c9]
data_only_2c9 = data_only_2c9.reset_index(drop=True)

To end, we use rdkit to extract a sdf containing the molecular structure for each set, as well as comparing the InchiKey of the generated molecule to the one available in PubChem

In [29]:
def get_sdf(df_molecules, out_file):
    w = Chem.SDWriter(out_file)
    consistent_inchikey = []
    RDKit_inchi = []
    for cid, smiles, inchikey in zip(df_molecules["PUBCHEM_CID"], df_molecules["CanonicalSMILES"], df_molecules["InChI"]):
        mol = Chem.MolFromSmiles(smiles)
        mol.SetProp("_Name", str(cid))
        w.write(mol)
        new_inchi = Chem.inchi.MolToInchi(mol)
        RDKit_inchi.append(new_inchi)
        consistent_inchikey.append(new_inchi==inchikey)
    df_molecules.loc[:,"Consistent InchiKey"] = consistent_inchikey
    df_molecules.loc[:,"RDKIT InchiKey"] = RDKit_inchi
    return df_molecules

In [41]:
data_shared = get_sdf(data_shared, "../datasets/CYP/shared_set_cyp.sdf")
data_only_3a4 = get_sdf(data_only_3a4, "../datasets/CYP/only_3a4_set_cyp.sdf")
data_only_2c9 = get_sdf(data_only_2c9, "../datasets/CYP/only_2c9_set_cyp.sdf")

In [42]:
data_shared.to_csv("../datasets/CYP/shared_set_cyp.csv", index=False)
data_only_3a4.to_csv("../datasets/CYP/only_3a4_set_cyp.csv", index=False)
data_only_2c9.to_csv("../datasets/CYP/only_2c9_set_cyp.csv", index=False)